# EasyVitessce Example: Select cells interactively, then use the list of cell IDs

## Import EasyVitessce

This import statement enables interactive plots by default.
Refer to the [EasyVitessce documentation](https://vitessce.github.io/easy_vitessce/) for how to disable interactive plotting or configure other behaviors.

In [ ]:
import easy_vitessce as ev

## Dependencies for datasets

In [ ]:
import scanpy as sc

## Load the example AnnData object

In [ ]:
adata = sc.datasets.pbmc68k_reduced()

## Plot the data

Store the return value of `sc.pl.plotting_function()` in a variable.
See more details at https://vitessce.github.io/easy_vitessce/advanced.html#access-the-vitessce-configuration

In [ ]:
vw = sc.pl.umap(adata, color="bulk_labels")
vw

## Get the set of lasso-ed cell IDs

First, lasso a set of cells in the user interface.

See more details at https://vitessce.github.io/easy_vitessce/advanced.html#access-values-from-the-coordination-space

In [ ]:
current_config = vw._config
obs_set_selection = current_config["coordinationSpace"]["obsSetSelection"]
additional_obs_sets = current_config["coordinationSpace"]["additionalObsSets"]

The value of `featureSelection[coordinationScope]` will be an array (to allow for multi-selection, which is not used here but is important for other plot types such as the dot plot).

In [ ]:
obs_set_selection["A"]

In [ ]:
cell_ids = []
first_selected_set_path = obs_set_selection["A"][0]
first_selected_set_path_group_name = first_selected_set_path[0] # e.g., "My Selections"
first_selected_set_path_set_name = first_selected_set_path[1] # e.g., "Selection 1"

# The additional_obs_sets is technically a tree data structure,
# so we need to traverse it.
for group_node in additional_obs_sets["A"]["tree"]:
    if group_node["name"] == first_selected_set_path_group_name:
        for set_node in group_node["children"]:
            if set_node["name"] == first_selected_set_path_set_name:
                # The items in set_node["set"] are tuples like
                # (cell_id, prediction_score) but only care about the cell_id values here.
                cell_ids = [arr[0] for arr in set_node["set"]]

In [ ]:
len(cell_ids)

### Subset the AnnData object to include only the selected cells

In [ ]:
selection_adata = adata[cell_ids, :].copy()

#### Plot the filtered AnnData object

In [ ]:
sc.pl.umap(selection_adata, color="bulk_labels")